In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
!ollama pull llama3.2

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest 
Error: pull model manifest: 500: {"errors":[{"code":"INTERNAL_ERROR","message":"internal error"}]}


In [3]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

# Initialize the OpenAI client for Ollama integration
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [15]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for Windows "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [5]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [8]:
def optimize_gpt(python):    
    stream = ollama_via_openai.chat.completions.create(model=MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [10]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
exec(pi)

Result: 3.141592658589
Execution Time: 22.903049 seconds


In [13]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <chrono>

using namespace std;

// Function to calculate the expression for a constant number of iterations
double calculate(double param1, double param2) {
    // Initialize result and iteration variables with sufficient precision
    long double result = 0.0;
    long i = 1;
    
    while (i <= 100000000_000ll) { // Use ll for large integers and ensure no overflow
        double j1 = i * param1 - param2;
        result -= 1 / j1; // Reduced precision due to limitations of floating-point representation
        double j2 = i * param1 + param2;
        result += 1 / j2;
        
        if ((j1 < 1e-30) || (j2 > 1e30)) { // Check for division by zero in floating point arithmetic 
            cout << "Warning: Division by zero encountered!" << endl; 
            exit(1);
        }
        
        i++;
    }

    return result;
}

int main() {
    auto start_time = chrono::high_resolution_clock::now(); // Use high-resolution clock
    double result = 

In [14]:
exec(pi)

Result: 3.141592658589
Execution Time: 22.992001 seconds


In [16]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

'clang++' is not recognized as an internal or external command,
operable program or batch file.
'.' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
# Repeat for Claude - again, use the right approach for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

'clang++' is not recognized as an internal or external command,
operable program or batch file.
'.' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [20]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 70.923048 seconds


In [21]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <random>

using namespace std;

// Mersenne Twister LCG generator class
class mersenne_twister {
public:
    unsigned int m, n, lower_mask, upper_mask, mt0, mt1, temp;
    vector<unsigned int> mix, a, t;

    void init(unsigned int _mt0, unsigned int _mt1) {
        mt0 = _mt0; 
        mt1 = _mt1;

        // Initialize mersenne-twister internal arrays
        s seed_256(256); //
        for(int i = 0; i != t.size(); ++i) {t[i] = s.nextInt() << ((i & 1)?8:16)}
        mix[t[0]] ^= mt0;

        for(int i = lower_mask + 1;i != n;++i){ 
            int i *= 2;
            temp = (mix[i] & t[mt0]) | (t[(mt0 += 1) % n] << (n-8)); 
            mix[i] = (mix[(i+1)&lower_mask] ^ temp);
        }
    }

    unsigned int next() {
        lower_mask = (n >> 2) - 1;

        // LCG iteration
        for(int i=0;i!=t.size();++i){ 
            if(i == upper_mask){ t[i] = (mix[(mt-i+1)%n] ^ t[i-1&upper_mask]) << 15; continue;} 

            unsi

In [25]:
def stream_gpt(python):    
    stream = ollama_via_openai.chat.completions.create(model=MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')


In [26]:
def optimize(python):
    result = stream_gpt(python)
    for stream_so_far in result:
        yield stream_so_far        


In [30]:
import gradio as gr

stop_flag = False  # Global flag for stopping generation

def optimize_with_stop(python):
    global stop_flag
    stop_flag = False  # Reset stop flag when new generation starts
    result = stream_gpt(python)  # Assuming this is your streaming function
    for stream_so_far in result:
        if stop_flag:
            break
        yield stream_so_far

def stop_generation():
    global stop_flag
    stop_flag = True  # Set stop flag to True when Stop button is clicked

with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        convert = gr.Button("Convert code")
        stop = gr.Button("Stop")

    convert.click(optimize_with_stop, inputs=[python], outputs=[cpp])
    stop.click(stop_generation)

ui.launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [31]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [32]:
import subprocess

def write_output(code):
    with open("optimized.cpp", "w") as f:
        f.write(code)

def execute_cpp(code):
    write_output(code)
    try:
        # Compile the C++ code using g++
        compile_cmd = ["g++", "-Ofast", "-std=c++17", "-o", "optimized.exe", "optimized.cpp"]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        
        # Run the compiled executable
        run_cmd = ["optimized.exe"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"



In [33]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [35]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    
    with gr.Row():
        convert = gr.Button("Convert code")
    
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    # Removed the model selection & directly using GPT
    convert.click(lambda code: optimize(code), inputs=[python], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch()


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\GAD\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\GAD\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\GAD\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2045, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\GAD\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1592, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\GAD\anaconda3\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_asyn